In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from hyperopt import hp
from hyperopt import tpe
from hyperopt import Trials
from hyperopt import fmin
from tqdm import tqdm_notebook

%matplotlib inline

In [2]:
df = pd.read_excel('lcap2.xlsx', index_col='Date')

In [3]:
def create_ema(series, lag=12):
    """
    Creates EMA for pd.Series with specified lag
    """
    tick=series.name
    dtf = pd.DataFrame(series)
    
    k = 2/(lag + 1)
    dtf['ma'] = dtf[tick].rolling(lag).mean()
    dtf['ema'] = dtf[tick]*k + dtf['ma'].shift(1)*(1-k)
    return dtf['ema']

def create_macd(series, lag1=12, lag2=26, signal_lag=9, start_current=1, comm=0.002):
    """
    Creates a pd.DataFrame with calculated MACD, signal line and calculates the strategy's return.
    It also asks for initial position: 1 for buy or -1 for sell
    """
    
    #Prepares names and a dataframe
    tick=series.name
    dtf = pd.DataFrame(series)
    emaFast = 'ema' + str(lag1)
    emaSlow = 'ema' + str(lag2)
    
    #Calculates EMA and MACD
    dtf[emaFast] = create_ema(series, lag=lag1)
    dtf[emaSlow] = create_ema(series, lag=lag2)
    dtf['macd'] = dtf[emaFast] - dtf[emaSlow]
    dtf['signal'] = create_ema(dtf['macd'], lag=signal_lag)
    
    #Identifies buy and sell signals and aggregates them into a single series
    dtf['buy'] = np.where( (dtf['macd'] > dtf['signal']) & (dtf['macd'].shift(1) < dtf['signal'].shift(1)), 1, 0 )
    dtf['sell'] = np.where( (dtf['macd'] < dtf['signal']) & (dtf['macd'].shift(1) > dtf['signal'].shift(1)), 1, 0 )
    dtf['bns'] = np.where( dtf['buy'] == 1, 1, 0 )
    dtf['bns'] = np.where( dtf['sell'] == 1, -1, dtf['bns'] )
    
    #Calculates current position - long or short the stock and sets initial condition 
    dtf['current'] = dtf['bns']
    dtf['current'].iloc[0] = start_current
    dtf['current'] = np.where(dtf['current'] == 0, np.nan, dtf['current'])
    dtf['current'] = dtf['current'].fillna(method='ffill')
    
    #Calculates the effect of MACD and applies commissions for every direction change
    dtf['chng'] = dtf[tick]/dtf[tick].shift(1)-1
    dtf['effect'] = 1+dtf['chng']*dtf['current'].shift(1)
    dtf['effect'] = dtf['effect'].fillna(1)
    dtf['effect'] = np.where(dtf['bns'] == 0, dtf['effect'], dtf['effect']-comm)
    
    #Calculates the return in terms of the stock price
    dtf['strategy'] = dtf['effect'].cumprod()*dtf[tick].iloc[0]
    
    return dtf


In [4]:
"""
The following checks MACD indicator for all of the stocks.
It trains on the first 90% of the data and checks on the remaining 10%
Results are aggregated into the 'ticker_results' dictionary
"""

train_data = 0.9
end_train = int(df.shape[0]*train_data)
attempts_to_find_maximum = 300

ticker_results = {}


for tick in tqdm_notebook(df.columns):
    
    #Creates a specific dataframe for that stock
    train_df = df[[tick]].iloc[:end_train]

    def calc_return(params, series=train_df[tick]):

        my_df = create_macd(series, lag1=int(params['lag1']), lag2=int(params['lag2']), \
                            signal_lag=int(params['signal_lag']))
        return -my_df['strategy'].iloc[-1]
    
    #Bayesian optimization process
    space_index = {
            'lag1': hp.quniform('lag1', 5, 200, 1),
            'lag2': hp.quniform('lag2', 10, 400, 1),
            'signal_lag': hp.quniform('signal_lag', 5, 100, 1)
        }

    tpe_algo = tpe.suggest
    tpe_trials = Trials()
    tpe_best = fmin(fn=calc_return, space=space_index, algo=tpe_algo, 
                    trials=tpe_trials, max_evals=attempts_to_find_maximum, verbose=False)
    
    #Testing process
    spy_test = create_macd(df[tick], lag1=int(tpe_best['lag1']), lag2=int(tpe_best['lag2']), \
                        signal_lag=int(tpe_best['signal_lag']))
    spy_test = spy_test.iloc[end_train:]
    spy_test['strategy'] = spy_test['effect'].cumprod()*spy_test[tick].iloc[0]
    
    abs_perf = spy_test['strategy'].iloc[-1]/spy_test['strategy'].iloc[0] - 1
    rel_perf = spy_test['strategy'].iloc[-1]/spy_test['strategy'].iloc[0] - spy_test[tick].iloc[-1]/spy_test[tick].iloc[0]
        
    ticker_results[tick] = [abs_perf, rel_perf]

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
ticker_results = pd.DataFrame(ticker_results, index=['abs_perf', 'rel_perf']).T
ticker_results['better'] = np.where(ticker_results.rel_perf > 0, 1, 0)
ticker_results['actual'] = df.iloc[-1]/df.iloc[-spy_test.shape[0]]

In [6]:
print("Average performance is", str(round(ticker_results.rel_perf.mean()*100, 2))+'% over/under buy-and-hold strategy')
print("MACD was better than buy-and-hold strategy for", ticker_results.better.sum(), \
      "(" + str(round(ticker_results.better.mean()*100,2)) + "%) of all stocks")
print("The correlation between MACD's outperformance and stock's actual performance is", \
     round(ticker_results[['rel_perf', 'actual']].corr().iloc[0][1], 2))
print("Equal weight result of MACD-driven investment for all stocks is:", \
     str(round(ticker_results.abs_perf.mean()*100, 2))+'% for the period between', \
     spy_test.index[0].date(), spy_test.index[-1].date())

Average performance is 10.28% over/under buy-and-hold strategy
MACD was better than buy-and-hold strategy for 216 (49.77%) of all stocks
The correlation between MACD's outperformance and stock's actual performance is -0.44
Equal weight result of MACD-driven investment for all stocks is: -2.77% for the period between 2019-05-02 2020-05-13
